# 📘 Agentic 架构 11：元控制器 (Meta-Controller)

欢迎来到我们系列的第十一个笔记本。今天，我们将构建一个**元控制器**（Meta-Controller），这是一种监督式代理架构，用于协调一组专长子代理。这种模式是创建强大、多才多艺 AI 系统的基石。

与其构建一个试图包办一切的单一、庞大代理，不如让元控制器充当智能调度器。它接收传入请求，分析请求的性质，并将其路由到可用代理池中最合适的专长代理。这使得每个子代理都能针对其特定任务进行高度优化，从而提升整体性能和模块化程度。

我们将通过构建一个包含三个不同专长代理的系统来演示这一架构：
1.  **通用代理：** 处理日常对话和简单问题。
2.  **研究代理：** 配备搜索工具，用于回答关于近期事件或复杂主题的问题。
3.  **编码代理：** 专注于生成 Python 代码片段的专长代理。

元控制器将成为整个系统的“大脑”，它会检查每个用户查询，并决定哪个代理最适合响应。这创造了一个灵活且易于扩展的系统，只需创建新的专长代理并让控制器了解它，即可添加新功能。

### 定义
**元控制器**（Meta-Controller，或称路由器）是一种多代理系统中的监督代理，负责分析传入任务并将其分派给合适的专长子代理或工作流程。它充当智能路由层，决定哪个工具或专家最适合处理当前任务。

### 高层工作流程

1.  **接收输入：** 系统接收用户请求。
2.  **元控制器分析：** 元控制器代理检查请求的意图、复杂度和内容。
3.  **分派给专长代理：** 根据分析结果，元控制器从预定义的代理池中选择最合适的专长代理（例如，“研究者”、“编码者”、“聊天机器人”）。
4.  **执行任务：** 被选中的专长代理执行任务并生成结果。
5.  **返回结果：** 专长代理的结果返回给用户。在更复杂的工作流程中，控制权可能返回到元控制器以进行后续步骤或监控。

### 何时使用 / 应用场景
*   **多服务 AI 平台：** 为提供多种服务的平台（如文档分析、数据可视化、创意写作）提供单一入口。
*   **自适应个人助理：** 能够在不同模式或工具之间切换的助理，例如管理日历、搜索网络或控制智能家居设备。
*   **企业工作流程：** 根据工单内容将客户支持工单路由到正确的部门（技术、计费、销售）。

### 优势与劣势
*   **优势：**
    *   **灵活性与模块化：** 通过简单添加新的专长代理并更新控制器的路由逻辑，即可极易扩展新功能。
    *   **性能：** 允许使用高度优化的专家代理，而不是一个样样通但样样松的万金油模型。
*   **劣势：**
    *   **控制器成为单点故障：** 整个系统的质量高度依赖控制器正确路由任务的能力。一次错误的路由决策会导致次优或错误的结果。
    *   **潜在延迟增加：** 与直接调用单一代理相比，额外的路由步骤可能会增加少量延迟。

## 第零阶段：基础与设置

我们将安装库并设置运行环境。我们需要 `langchain-tavily` 来为研究代理提供搜索工具。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

In [2]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_tavily import TavilySearch
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Meta-Controller (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：构建专长代理

首先，我们将创建我们的专家代理团队。每个代理都是一个简单的链，具有特定的角色设定，对于研究代理，还会配备工具。我们将把它们封装成节点函数，以便在 LangGraph 中使用。

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=3)

# Define the state for the overall graph
class MetaAgentState(TypedDict):
    user_request: str
    next_agent_to_call: Optional[str]
    generation: str

# A helper factory function to create specialist agent nodes
def create_specialist_node(persona: str, tools: list = None):
    """Factory to create a specialist agent node."""
    system_prompt = f"You are a specialist agent with the following persona: {persona}. Respond directly and concisely to the user's request based on your role."
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{user_request}")
    ])
    
    if tools:
        chain = prompt | llm.bind_tools(tools)
    else:
        chain = prompt | llm
        
    def specialist_node(state: MetaAgentState) -> Dict[str, Any]:
        result = chain.invoke({"user_request": state['user_request']})
        return {"generation": result.content}
    
    return specialist_node

# 1. Generalist Agent Node
generalist_node = create_specialist_node(
    "You are a friendly and helpful generalist AI assistant. You handle casual conversation and simple questions."
)

# 2. Research Agent Node
research_agent_node = create_specialist_node(
    "You are an expert researcher. You must use your search tool to find information to answer the user's question.",
    tools=[search_tool]
)

# 3. Coding Agent Node
coding_agent_node = create_specialist_node(
    "You are an expert Python programmer. Your task is to write clean, efficient Python code based on the user's request. Provide only the code, wrapped in markdown code blocks, with minimal explanation."
)

print("Specialist agents defined successfully.")

Specialist agents defined successfully.


## 第二阶段：构建元控制器

这是我们系统的“大脑”。元控制器是一个由 LLM 驱动的节点，它唯一的职责就是决定将请求路由到哪个专长代理。其提示的质量对整个系统的性能至关重要。

In [4]:
# Pydantic model for the controller's routing decision
class ControllerDecision(BaseModel):
    next_agent: str = Field(description="The name of the specialist agent to call next. Must be one of ['Generalist', 'Researcher', 'Coder'].")
    reasoning: str = Field(description="A brief reason for choosing the next agent.")

def meta_controller_node(state: MetaAgentState) -> Dict[str, Any]:
    """The central controller that decides which specialist to call."""
    console.print("--- 🧠 Meta-Controller Analyzing Request ---")
    
    # Define the specialists and their descriptions for the controller
    specialists = {
        "Generalist": "Handles casual conversation, greetings, and simple questions.",
        "Researcher": "Answers questions about recent events, complex topics, or anything requiring up-to-date information from the web.",
        "Coder": "Writes Python code based on a user's specification."
    }
    
    specialist_descriptions = "\n".join([f"- {name}: {desc}" for name, desc in specialists.items()])
    
    prompt = ChatPromptTemplate.from_template(
        f"""You are the meta-controller for a multi-agent AI system. Your job is to analyze the user's request and route it to the most appropriate specialist agent.

Here are the available specialists:
{specialist_descriptions}

Analyze the following user request and choose the best specialist to handle it. Provide your decision in the required format.

User Request: "{{user_request}}""""
    )
    
    controller_llm = llm.with_structured_output(ControllerDecision)
    chain = prompt | controller_llm
    
    decision = chain.invoke({"user_request": state['user_request']})
    console.print(f"[yellow]Routing decision:[/yellow] Send to [bold]{decision.next_agent}[/bold]. [italic]Reason: {decision.reasoning}[/italic]")
    
    return {"next_agent_to_call": decision.next_agent}

print("Meta-Controller node defined successfully.")

Meta-Controller node defined successfully.


## 第三阶段：组装并运行图

现在，我们将使用 LangGraph 将所有组件连接起来。图将从元控制器开始，然后根据其决策，通过条件边将状态路由到正确的专长代理节点。专长代理运行完成后，图将结束。

In [5]:
# Build the graph
workflow = StateGraph(MetaAgentState)

# Add nodes for the controller and each specialist
workflow.add_node("meta_controller", meta_controller_node)
workflow.add_node("Generalist", generalist_node)
workflow.add_node("Researcher", research_agent_node)
workflow.add_node("Coder", coding_agent_node)

# Set the entry point
workflow.set_entry_point("meta_controller")

# Define the conditional routing logic
def route_to_specialist(state: MetaAgentState) -> str:
    """Reads the controller's decision and returns the name of the node to route to."""
    return state["next_agent_to_call"]

workflow.add_conditional_edges(
    "meta_controller",
    route_to_specialist,
    {
        "Generalist": "Generalist",
        "Researcher": "Researcher",
        "Coder": "Coder"
    }
)

# After any specialist runs, the process ends
workflow.add_edge("Generalist", END)
workflow.add_edge("Researcher", END)
workflow.add_edge("Coder", END)

meta_agent = workflow.compile()
print("Meta-Controller agent graph compiled successfully.")

Meta-Controller agent graph compiled successfully.


## 第四阶段：演示

让我们使用各种不同的提示来测试我们的元控制器，看看它是否能正确地将它们分派给合适的专长代理。

In [6]:
def run_agent(query: str):
    result = meta_agent.invoke({"user_request": query})
    console.print("\n[bold]Final Response:[/bold]")
    console.print(Markdown(result['generation']))

# Test 1: Should be routed to the Generalist
console.print("--- 💬 Test 1: General Conversation ---")
run_agent("Hello, how are you today?")

# Test 2: Should be routed to the Researcher
console.print("\n--- 🔬 Test 2: Research Question ---")
run_agent("What were NVIDIA's latest financial results?")

# Test 3: Should be routed to the Coder
console.print("\n--- 💻 Test 3: Coding Request ---")
run_agent("Can you write me a python function to calculate the nth fibonacci number?")

--- 💬 Test 1: General Conversation ---


--- 🧠 Meta-Controller Analyzing Request ---
Routing decision: Send to Generalist. Reason: The user's request is a simple greeting, which falls under the category of casual conversation handled by the Generalist agent.



Final Response:


Hello there! How can I help you today?


--- 🔬 Test 2: Research Question ---


...

NVIDIA's latest financial results, for the quarter ending in April 2024, were exceptionally strong. They reported revenue of $26.04 billion, a significant increase year-over-year, driven largely by their Data Center revenue which hit a record $22.6 billion. Their GAAP earnings per diluted share were $5.98.


--- 💻 Test 3: Coding Request ---


--- 🧠 Meta-Controller Analyzing Request ---
Routing decision: Send to Coder. Reason: The user is explicitly asking for a Python function, which is a coding task. The Coder agent is the specialist for this.



Final Response:


```python
def fibonacci(n):
    """Calculates the nth Fibonacci number."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n + 1):
            a, b = b, a + b
        return b
```

## 结论

在本笔记本中，我们成功实现了**元控制器**架构。我们的测试清楚地展示了其核心功能：充当一个智能且动态的路由器。

1.  简单的问候被正确识别并发送给**通用代理**。
2.  关于近期金融新闻的查询被分派给**研究代理**，后者使用其搜索工具获取了最新信息。
3.  请求代码片段的需求被路由到**编码代理**，后者提供了格式良好且正确的函数。

这种模式在构建可扩展且易维护的 AI 系统时具有异常强大的能力。通过分离关注点，每个专长代理都可以独立改进，而不会影响其他代理。系统的整体智能只需通过添加新的、更强大的专长代理并让元控制器了解它们即可得到提升。虽然控制器本身可能成为潜在瓶颈，但它作为灵活编排者的角色，是高级代理式设计中的基石。